### Prueba 3

In [ ]:
import os
import time
import logging
import wikipedia
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from openai import BadRequestError 


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("agente_investigador.log"), 
        logging.StreamHandler() 
    ]
)
logger = logging.getLogger(__name__)


MI_URL_BASE = "https://models.inference.ai.azure.com"
MI_TOKEN = "XXXXXXXXXX"
MI_MODELO = "gpt-4o"

os.environ["OPENAI_API_BASE"] = MI_URL_BASE
os.environ["OPENAI_API_KEY"] = MI_TOKEN
os.environ["OPENAI_MODEL_NAME"] = MI_MODELO

logger.info(f"Sistema iniciado. Modelo configurado: {MI_MODELO}")

class SecurityManager:
    """
    Encargado de validar inputs y asegurar que el agente actúe dentro de normas éticas.
    """
    def __init__(self):
        self.forbidden_terms = [
            "hackear","ilegal", "delito", "malware", 
            "explotar vulnerabilidad", "bomba", "arma", "explosivo", 
            "atentado", "muerte", "suicidio", "veneno", "terrorismo", 
            "narcotráfico", "secuestro"
        ]

    def validate_input(self, user_input: str) -> bool:
        """Valida que el input sea seguro y ético."""
        if not user_input or len(user_input.strip()) < 2:
            logger.warning("Input rechazado: Demasiado corto o vacío.")
            return False
        
        normalized_input = user_input.lower()
        
        for term in self.forbidden_terms:
            if term in normalized_input:
                logger.warning(f"Busca rechazada por terminos ed seguridad. Término detectado: '{term}'")
                return False
        
        return True

class WikipediaSearchTool(BaseTool):
    name: str = "BuscadorWikipedia"
    description: str = "Busca en Wikipedia un tema y devuelve un resumen detallado."
    
    def _run(self, query: str) -> str:
        logger.info(f"Herramienta Wikipedia invocada con query: {query}") 
        try:
            wikipedia.set_lang("es")
            summary = wikipedia.summary(query, sentences=5)
            return summary
        except wikipedia.exceptions.PageError:
            msg = f"No se encontró página para: {query}"
            logger.error(msg)
            return msg
        except wikipedia.exceptions.DisambiguationError as e:
            msg = f"Búsqueda ambigua para {query}"
            logger.warning(msg)
            return f"{msg}. Opciones: {e.options[:3]}"
        except Exception as e:
            logger.critical(f"Error crítico en Wikipedia Tool: {str(e)}")
            return f"Error: {str(e)}"


class ResearchAssistantSystem:
    def __init__(self):
        self.wikipedia_tool = WikipediaSearchTool()
        self.security = SecurityManager()
        self.interaction_counter = 0 
        self._setup_agents()

    def _setup_agents(self):
        # agentes
        self.investigador = Agent(
            role="Investigador",
            goal="Encontrar información relevante y precisa.",
            backstory="Experto en buscar datos y filtrar informacion.",
            tools=[self.wikipedia_tool],
            verbose=True,
            allow_delegation=False
        )

        self.redactor = Agent(
            role="Redactor Profesional",
            goal="Escribir informes claros y concretos.",
            backstory="Escritor experto en transformar datos en reportes completos.",
            verbose=True,
            allow_delegation=False
        )

    def process_request(self, topic: str):
        """
        Orquesta el flujo completo: Validación -> Ejecución -> Métricas
        """
        self.interaction_counter += 1
        request_id = f"REQ-{self.interaction_counter}"
        
        logger.info(f"[{request_id}] Iniciando procesamiento para tema: '{topic}'")

        is_valid = self.security.validate_input(topic)
        if not is_valid:
            return "Solicitud rechazada: El tema que quieres buscar viola nuestra politica de seguridad."

        # 2. Definición de Tareas
        research_task = Task(
            description=f"Investiga: '{topic}'. Encuentra puntos clave y hechos.",
            expected_output="Documento con puntos clave.",
            agent=self.investigador
        )

        write_task = Task(
            description="Redacta un informe completo en markdown basado en la investigación.",
            expected_output="Informe final en markdown.",
            agent=self.redactor,
            context=[research_task]
        )

        crew = Crew(
            agents=[self.investigador, self.redactor],
            tasks=[research_task, write_task],
            process=Process.sequential,
            verbose=True
        )

        try:
            start_time = time.time()             
            result = crew.kickoff() 
            end_time = time.time() 
            duration = end_time - start_time            
            logger.info(f"[{request_id}] Completado exitosamente.")
            logger.info(f"[{request_id}] Latencia de ejecución: {duration:.2f} segundos")
            
            return result

        except BadRequestError as e:
            logger.warning(f"[{request_id}] Bloqueo por Política de Contenidos del Modelo (Azure/OpenAI): {e}")
            return "La solicitud fue procesada pero rechazada por el sistema de seguridad."

        except Exception as e:            
            logger.error(f"[{request_id}] Fallo técnico no controlado: {e}")
            return f"Ocurrió un error técnico procesando su solicitud. Consulte los logs para más detalles."

if __name__ == "__main__":
    system = ResearchAssistantSystem()
    
    print("\nSISTEMA DE INVESTIGACIÓN SEGURO Y OBSERVABLE")
    print("   (Escribe 'salir' para terminar)")

    while True:
        try:
            topic = input("\n¿Sobre qué tema deseas investigar hoy?: ")
            
            
            
            if topic.lower() in ["salir", "exit", "quit"]:
                logger.info("Sesión finalizada.")
                print("¡Hasta la próxima!")
                break

            response = system.process_request(topic)

            print("\n" + "="*40)
            print("RESULTADO FINAL")
            print("="*40)
            print(response)
            print("="*40)

        except KeyboardInterrupt:
            logger.info("Interrupción de teclado detectada. Cerrando.")
            print("\nCierre forzado.")
            break
        except Exception as e:
            logger.critical(f"Error no manejado en el loop principal: {e}")

2025-11-24 15:22:05,728 - INFO - Sistema iniciado. Modelo configurado: gpt-4o



SISTEMA DE INVESTIGACIÓN SEGURO Y OBSERVABLE
   (Escribe 'salir' para terminar)


2025-11-24 15:22:10,656 - INFO - [REQ-1] Iniciando procesamiento para tema: 'como hacer una bomba'
2025-11-24 15:22:10,657 - WARNING - Busca rechazada por terminos ed seguridad. Término detectado: 'bomba'



RESULTADO FINAL
Solicitud rechazada: El tema que quieres buscar viola nuestra politica de seguridad.


2025-11-24 15:22:15,344 - INFO - [REQ-2] Iniciando procesamiento para tema: 'quien es spiderman?'


...

2025-11-24 15:22:38,894 - INFO - [REQ-2] Completado exitosamente.
2025-11-24 15:22:38,904 - INFO - [REQ-2] Latencia de ejecución: 23.54 segundos



RESULTADO FINAL
```markdown
# Informe sobre Spider-Man

## Origen y Creación  

Spider-Man es un superhéroe emblemático originario de los cómics estadounidenses publicados por Marvel Comics. Su creación se atribuye al guionista y editor Stan Lee y al artista Steve Ditko. El personaje realizó su debut en la antología *Amazing Fantasy* #15 en agosto de 1962, durante la Edad de Plata del Cómic, marcando el inicio de una de las trayectorias más icónicas en la historia del entretenimiento.

## Popularidad  

Considerado como uno de los superhéroes más icónicos y exitosos comercialmente, Spider-Man ha trascendido al universo de los cómics, apareciendo en diversos formatos que incluyen películas, series de televisión, videojuegos, novelas y hasta producciones teatrales. Su impacto y conexión con la audiencia lo han convertido en un símbolo cultural reconocido a nivel global.

## Razón de su Creación  

Spider-Man fue creado en respuesta al interés creciente del público adolescente en los cóm

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d8c37274-49ed-415b-aa45-3d5686485cc5                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Informe sobre Spider-Man                                                                                     │
│                                                                                                                 │
│  ## Origen y Creación                                                                                           │
│                                                                                                                 │
│  Spider-Man es un superhéroe emblemático originario de los cómics estadounidenses publicados por Marvel         │
│  Comics. Su creación se atribuye al guionista y editor Stan Lee y al artista Steve Ditko. El personaje realizó  │
│  su debut en la antología *Amazing Fantasy* #15 en agosto de 1962, durante la Edad de Plata del Cómic,          │
│  marcando el inicio de una de las trayectorias más icónicas en la historia del entretenimiento.                 │
│                                                                                                                 │
│  ## Popularidad                                                                                                 │
│                                                                                                                 │
│  Considerado como uno de los superhéroes más icónicos y exitosos comercialmente, Spider-Man ha trascendido al   │
│  universo de los cómics, apareciendo en diversos formatos que incluyen películas, series de televisión,         │
│  videojuegos, novelas y hasta producciones teatrales. Su impacto y conexión con la audiencia lo han convertido  │
│  en un símbolo cultural reconocido a nivel global.                                                              │
│                                                                                                                 │
│  ## Razón de su Creación                                                                                        │
│                                                                                                                 │
│  Spider-Man fue creado en respuesta al interés creciente del público adolescente en los cómics durante los      │
│  años 60, una época marcada por el éxito de los 4 Fantásticos. Al abordar los problemas y emociones comunes de  │
│  los jóvenes, el personaje resonó profundamente con esta audiencia, estableciendo un modelo único de            │
│  superhéroe identificable.                                                                                      │
│                                                                                                                 │
│  ## Serie Inicial                                                                                               │
│                                                                                                                 │
│  El éxito de su debut condujo al lanzamiento de su propia serie individual, titulada *The Amazing Spider-Man*,  │
│  que comenzó a publicarse en marzo de 1963. Esta serie cimentó las bases de su historia y lo consagró como uno  │
│  de los personajes más importantes de Marvel Comics.  

2025-11-24 15:22:46,739 - INFO - Sesión finalizada.


¡Hasta la próxima!
